In [2]:
# 初始化环境
import os

os.environ['OPENAI_BASE_URL'] = 'https://key.wenwen-ai.com/v1'
os.environ['OPENAI_API_KEY'] = 'sk-6V2exWFBSa2lmuZ7C0D773D1BaEd4fB7A1B6A0A265D550C6'

In [3]:
# tools 示例

from openai import OpenAI
import json

def custom_encoder(obj):
    if hasattr(obj, 'model_dump'):
        return obj.model_dump()  # 调用自定义的 to_dict 方法
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")


# 1. 定义函数
# 1.1 定义模拟获取天气的本地函数
def get_current_weather(location, unit):
    # Call the weather API
    return f"It's 20 {unit} in {location}"


# 1.2 定义函数字典方便调用
function_dict = {
    "get_current_weather": get_current_weather,
}

# 1.3 定义chat接口需要的函数
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]

# 2. 第一次调用chat接口，返回的是函数调用的提示
messages = [
    {"role": "user", "content": "What's the weather like in Boston today with celsius and fahrenheit?"}]

client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

print(f"first call: {completion.model_dump_json()}")

# 3. 从结果接口的结果中获取函数调用的参数 进行本地函数调用
# 3.1 获取函数调用的参数
response_message = completion.choices[0].message
messages.append(response_message)
# 遍历tool_calls，获取函数调用的id，函数名，函数参数
for tool_call in response_message.tool_calls:
    tool_id = tool_call.id
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    # 3.2 调用本地函数
    function_response = function_dict.get(function_name)(**function_args)
    # 3.3 将本地函数的结果作为chat接口的输入
    messages.append({
        "role": "tool",
        "tool_call_id": tool_id,
        "content": function_response,
    })

print(f"second call massages: {json.dumps(messages, default=custom_encoder,ensure_ascii=False)}")

# 4. 第二次调用chat接口，返回的是chat的最终结果
completion_final = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(f"second call: {completion_final.model_dump_json()}")

first call: {"id": "chatcmpl-ByyNVVsQ7hOWVDZEIOS5qRJ3F6QtQ", "choices": [{"finish_reason": "tool_calls", "index": 0, "logprobs": null, "message": {"content": null, "refusal": null, "role": "assistant", "annotations": [], "audio": null, "function_call": null, "tool_calls": [{"id": "call_KNUcRgduS8Icy4lClmmqOKKl", "function": {"arguments": "{\"location\": \"Boston, MA\", \"unit\": \"celsius\"}", "name": "get_current_weather"}, "type": "function"}, {"id": "call_8Pl4NGb0Qp2qt6ZJNuUWkZ0i", "function": {"arguments": "{\"location\": \"Boston, MA\", \"unit\": \"fahrenheit\"}", "name": "get_current_weather"}, "type": "function"}]}}], "created": 1753871473, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_ee1d74bde0", "usage": {"completion_tokens": 61, "prompt_tokens": 86, "total_tokens": 147, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_